In [1]:
# Import the Twython class
from twython import TwythonStreamer
import csv
from twython import Twython
import json
import pandas as pd
import os

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [2]:
# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):
    list_tweets = []
        
    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.list_tweets.append(tweet_data)
            self.save_to_json(self.list_tweets)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        #self.disconnect() #DESCOMENTAR PARA PARAR DE COLETAR QUANDO DER ERRO
    
    # Save each tweet to csv file
    def save_to_json(self, tweet):
        with open("saved_tweets.json", "wt") as file:
            json.dump(tweet, file, indent = 4)

In [3]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
os.remove("saved_tweets.json")
stream.statuses.filter(track='NBA')

KeyboardInterrupt: 